In [3]:
!pip install opencv-python

import cv2
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

In [6]:
!pip install opencv-python --upgrade


In [52]:
CONFIDENCE = 0.5
SCORE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5
config_path = "obj_detect/yolov3.cfg"
weights = "obj_detect/yolov3.weights"
labels = open("obj_detect/coco.names").read().strip().split("\n")
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

In [53]:
net = cv2.dnn.readNetFromDarknet(config_path, weights)


In [70]:

# Load the pre-trained YOLO model and its configuration
net = cv2.dnn.readNet("obj_detect/yolov3.cfg", "obj_detect/yolov3.weights")

# Assuming you have a list of output layer names
output_layer_names = net.getUnconnectedOutLayersNames()

def model_output(frame):
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)

    # Use the list of output layer names
    output_blobs = net.forward(output_layer_names)

    boxes, confidences, class_ids = [], [], []

    for output in output_blobs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > CONFIDENCE:
                box = detection[0:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confidences, class_ids

CONFIDENCE = 0.5

# Open the video file
video_path = "obj_detect/video.mp4"
video_capture = cv2.VideoCapture(video_path)

while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    boxes, confidences, class_ids = model_output(frame)

    # Draw bounding boxes and labels on the frame
    for i in range(len(boxes)):
        x, y, w, h = boxes[i]
        label = f"Class {class_ids[i]}, Confidence: {confidences[i]:.2f}"
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the frame
    cv2.imshow("Object Detection", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
def detection_recognition(path_name):
    image = cv2.imread(path_name)
    boxes, confidences, class_ids = model_output(path_name)
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)
    font_scale = 1
    thickness = 1
    if len(idxs)>0:
        for i in idxs.flatten():
            x,y = boxes[i][0], boxes[i][1]
            w,h = boxes[i][2], boxes[i][3]
            color = [int(c) for c in colors[class_ids[i]]]
            cv2.rectangle(image, (x,y), (x+w, y+h), color = color, thickness= thickness)
            text = f"{labels[class_ids[i]]}: {confidences[i]:.2f}"
            (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
            text_offset_x = x
            text_offset_y = y - 5
            box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
            overlay = image.copy()
            cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
            image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)#add two copies to one and the numbers define blending percent
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=font_scale, color=(0, 0, 0), thickness=thickness)
    %matplotlib inline
    plt.rcParams['figure.figsize'] = (10.0, 10.0)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

In [1]:
detection_recognition("obj_detect/video.mp4")

NameError: name 'detection_recognition' is not defined